In [ ]:
import opendatasets as od
import os
import PIL.Image
from PIL import Image
from PIL.ExifTags import TAGS

username = "verniereloic"
key = "03e78f24e94d43d80b595934fc372ecf"

od.download("https://www.kaggle.com/ayklovettc/images-with-exif")



## Extract exif in python 

We will not use id for next steps

In [ ]:
import opendatasets as od
import os
import PIL.Image
from PIL import Image
from PIL.ExifTags import TAGS


def get_meta():
    path = "Images/"
    folders = os.listdir("Images")
    categories = []

    for folder in folders:
        categories.append(folder)
        for image in os.listdir(path + folder):
            if image.endswith(".jpg"):
                img = PIL.Image.open(path + folder + "/" + image)
                exif_data = img.getexif()
                print(exif_data)

In [43]:
from __future__ import print_function
import binascii
import struct
from PIL import Image
import numpy as np
import scipy
import scipy.misc
import scipy.cluster
import json


def json_imgdata(imagename, cpt, category, color):
    data = { imagename : [ {'Id': cpt,'Category': category, 'Color': color}]}
    with open('json_data.json', 'a') as outfile:
        outfile.write(json.dumps(data, indent=4))
        outfile.write(",")
        outfile.close()


def find_color(NUM_CLUSTERS):

    path = "Images/"
    folders = os.listdir("Images")
    categories = []
    cpt = 0

    for folder in folders:
        categories.append(folder)
        for image in os.listdir(path + folder):
            cpt = cpt + 1
            if image.endswith(".jpg"):
                img = Image.open(path + folder + "/" + image)
                img = img.resize((150, 150)).convert("RGB") #To converte all image to RGB image
                ar = np.asarray(img)
                shape = ar.shape
                ar = ar.reshape(np.product(shape[:2]), shape[2]).astype(float)
                codes, dist = scipy.cluster.vq.kmeans(ar, NUM_CLUSTERS)
                vecs, dist = scipy.cluster.vq.vq(ar, codes)         # assign codes
                counts, bins = np.histogram(vecs, len(codes))    # count occurrences
                index_max = np.argmax(counts) # find most frequent
                peak = codes[index_max]
                colour = binascii.hexlify(bytearray(int(c)for c in peak)).decode('ascii')
                #print('Most frequent is %s (#%s)' % (peak, colour))
                json_imgdata(image, cpt, folder, colour) #insert data in a json file


find_color(5)
